In [1]:
import psycopg2
import requests
import nbimporter
import Init_database as indb

Importing Jupyter notebook from Init_database.ipynb


In [70]:
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="169.229.157.40"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [68]:
try:
    cur = conn.cursor()
    stmt1 = "select ST_AsText(location) from tbl_app_eachstations"
    cur.execute(stmt1)
    station_loc1 = cur.fetchall()
    stmt2 = "select ST_X(location), ST_Y(location) from tbl_app_eachstations"
    cur.execute(stmt2)
    station_loc2 = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(station_loc1[0])
print(station_loc2[0])

('POINT(116.26327 39.863907)',)
(116.26327, 39.863907)


In [14]:
# #------- Request stationid which has user activities from db -------#
try:
    cur = conn.cursor()
    cur.execute("select distinct stationid from tbl_app_useractivity;")
    stationWithAct_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print("Number of stations with user activites: ", len(stationWithAct_list))
# #------- Request pilecode from db for states -------#
staid_list = []
for staid in stationWithAct_list:
    staid_list.append(staid[0])
try:
    cur = conn.cursor()
    sta_list = ['2b3a35d4-cb94-4b8f-a47a-bbb054754066', '1d02640a-7f14-4fc3-baaf-60754a38fff7',
                'ac6cb946-5fff-491b-859e-55cc5f701657', 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66',
                'b134a159-16bd-4f5d-af1b-cd42599a3bab', '33a3c031-0a3a-417d-ba1b-53549bb08dc2', 
                'e4758edb-1183-4c51-887b-a52ca7abef3d', '13bdab71-110b-4448-a3c4-5c3113183546']
    #1. beijing hangkong jingmi jixie yanjiusuo chongdianzhan --'2b3a35d4-cb94-4b8f-a47a-bbb054754066',
    #2. beijing xijiao bingguan chongdianzhan -- '1d02640a-7f14-4fc3-baaf-60754a38fff7
    #3. beijing yangguanglicheng jiudian chongdianzhan --'ac6cb946-5fff-491b-859e-55cc5f701657',
    #4. beijing rongtao gongsi chongdianzhan -- '32d317a9-3d0c-4458-8357-89a2b3b1cf94'
    #5. beijing huairou qu yanshui lu -- 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'
    
    #6. shenzhen hengjiang dasha chongdianzhan -- 'b134a159-16bd-4f5d-af1b-cd42599a3bab'
    #7. shenzhen yinxing gongyeyuan chongdianzhan -- '33a3c031-0a3a-417d-ba1b-53549bb08dc2'
    #8. shenzhe wanke jiulongshan chongdianzhan -- 'e4758edb-1183-4c51-887b-a52ca7abef3d'
    #9. shenzhen jingyuan dasha chongdianzhan -- '13bdab71-110b-4448-a3c4-5c3113183546'
    stmt = "select pilecode from tbl_APP_pilecode where staid in %s"
    cur.execute(stmt, (tuple(sta_list),))
    
#     stmt = "select pilecode from tbl_APP_pilecode where staid = 'dd4a3ddc-e1b0-4e2e-b7ca-faef627f4e66'"
#     cur.execute(stmt)
#     cur.execute("select pilecode from tbl_APP_pilecode")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print("Nubmer of pills: ", len(pillcode_list))

('Number of stations with user activites: ', 3899)
('Nubmer of pills: ', 86)


In [28]:
pillcode_list[0][0]

'1101080330203'

In [50]:
# #------- Print attributes of charging piles -------#
try:
    cur = conn.cursor()
    stmt = "select column_name from INFORMATION_SCHEMA.COLUMNS where table_name = '{}'".format('tbl_app_piles')
    cur.execute(stmt)
    pill_attr = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
# for _ in pill_attr:
#     print(_[0])

# Create a charing station object

In [82]:
class station(object):
    def __init__(self, stationid, score, pricescore, servicescore, 
                 stationname, stationaddress, location, num_pills):
        self.id = stationid
        self.score = score
        self.pricescore = pricescore
        self.servicescore = servicescore
        self.name = stationname
        self.addr = stationaddress
        self.GPS = location
        self.pills = num_pills
    def update_numPills():
        self.pills += 1
    def delete_numPills():
        if self.pills >= 1:
            self.pills -= 1

In [ ]:
class pill(station):
    def __init__(self, pillcode, voltage, description, isfast, chargingport, power, pricedesc, prices, pricesize):
        self.pillcdoe = pillcode
        self.voltage = voltage
        self.description = description
        self.isfast = isfast
        self.chargingport = chargingport
        self.power = power
        self.pricedesc = pricedesc
        sefl.prices = prices
        sefl.pricesize = pricesize
    

# Create a charging pill object

# One pill daily utilization data from db

In [77]:
# #------- Request one pill daily utilization data from db -------#
try:
    cur = conn.cursor()
    stmt = "select pillcode, statecode, statename, time from tbl_APP_state where pillcode='{}'".format(pillcode_list[0][0])
    cur.execute(stmt)
    ult_data = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
# #------- Request the pill details from db -------#
try:
    cur = conn.cursor()
    stmt = "select voltage, description, isfast, chargingport, power, pricedesc, prices, pricesize from tbl_app_piles where pillcode='{}'".format(pillcode_list[0][0])
    cur.execute(stmt)
    pill_details = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)

    # #------- Request the station details from db -------#
try:
    cur = conn.cursor()
    stmt = "select servicescore, score, pricescore, stationname, stationaddress, ST_X(location), ST_Y(location) from tbl_app_eachstations where stationid='{}'".format(sta_list[0])
    cur.execute(stmt)
    sta_details = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)

In [80]:
for _ in pill_details[0]:
    print(_)

200V-500V
00:00~24:00:0.800000:00~24:00:1.1000
Y
国标2011 国标2015 
15kW-90kW
电费:00:00~24:00:1.1000;服务费:00:00~24:00:0.8000
[{u'servicePrice': u'0.8000', u'timeRange': u'00:00-24:00', u'electricPrice': u'1.1000', u'startTime': u'00:00'}]
1


In [79]:
for _ in sta_details[0]:
    print(_)

--
--
--
北京航空精密机械研究所充电站
北京市市辖区丰台区南苑东路5号
116.407528
39.806932
